# OpenHPC環境の削除

---

構築したOpenHPC環境を削除します。

## VCP の初期化
次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンで VCP を初期化します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

##  削除する UnitGroup の指定

削除する UnitGroup 名を指定してください。

In [ ]:
ugroup_name = 'TFclusterGPU'

###  操作可能な UnitGroup 名の表示（必要に応じて実行してください）

他の UnitGroup を削除したい場合は、次のセルを実行して UnitGroup 名を参照し、上のセルの ugroup_name を変更して実行してください。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

## 構築環境の削除

### OpenHPC 環境の削除

#### VCノードの削除

現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

現在のVCノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

まず計算ノード用VCノードを削除します。

In [ ]:
if ug.get_unit('compute'):
    ug.delete_units('compute', force=True)

マスターノードと UnitGroup の削除を行います。

In [ ]:
ug.cleanup()

削除後の UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

#### VCディスクの削除

NFS用のVCディスクを削除します。

> VCディスクを作成していない場合は、何もしません。

現在の状態を確認します。

In [ ]:
from IPython.display import display
ug_disk = vcp.get_ugroup(ugroup_name + '_disk')
if ug_disk:
    display(ug_disk.df_nodes())

VCディスクを削除します。

In [ ]:
if ug_disk:
    ug_disk.cleanup()

削除後のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

### Ansible設定のクリア

#### group_varsファイルのリネーム

In [ ]:
!mv group_vars/{ugroup_name}.yml group_vars/{ugroup_name}.yml.bak

#### インベントリから UnitGroup に対応するグループを削除

In [ ]:
import yaml
from pathlib import Path

inventory_path = Path('inventory.yml')

if inventory_path.exists():
    !cp {str(inventory_path)} {str(inventory_path)}.bak

    with inventory_path.open() as f:
        inventory = yaml.safe_load(f)

    if ugroup_name in inventory['all']['children']:
        del(inventory['all']['children'][ugroup_name])

    with inventory_path.open(mode='w') as f:
        f.write(yaml.safe_dump(inventory))

    !cat {str(inventory_path)}